In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
#import cubic_spline_planner

N_IND_SEARCH = 10  # Search index number
show_animation = True

In [2]:
!rm -f Data_Processor.pyc
Remove=True
if Remove==True:
    !rm -r Fig_iLQR
    !rm -r Rec_iLQR
    !rm -r Rec_iLQR_Raw

In [3]:
import numpy as np
import math
from copy import copy, deepcopy
TARGET_SPEED = 20# / 3.6  # [m/s] target speed
MAX_TIME=1500
T=5
NX=4
NU=2
DT=0.2
WB=50
MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 25#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -25#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 5.0  # maximum accel [m/ss]
    


In [4]:
from Motion import Motion,State
R = np.diag([10.0, 10.0])  # input cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix


def np_quad_form(a,b):
        res=a @ b @ a.T
        return res
    
def calc_cost( x, u,xref,t):
        
        
        loss = np_quad_form(u, R) + np_quad_form(xref - x, Q)
        
        loss_x1=2*Q[0,0]*(xref[0]-x[0])*(-1)
        loss_x2=2*Q[1,1]*(xref[1]-x[1])*(-1)
        loss_x3=2*Q[2,2]*(xref[2]-x[2])*(-1)
        loss_x4=2*Q[3,3]*(xref[3]-x[3])*(-1)
            
        loss_x=np.array([loss_x1,loss_x2,loss_x3,loss_x4])
        loss_xx=2 * Q
        
        Qdiag=np.array([Q[0,0], Q[1,1], Q[2,2], Q[3,3]],dtype=np.float64)
        error=np.array(xref-x)
        
        loss_u1 = 2. * u[0] * R[0,0] #+ np.sum(2*Qdiag*error*(-1)*_B[:,0])
        loss_u2 = 2. * u[1] * R[1,1] #+ np.sum(2*Qdiag*error*(-1)*_B[:,1])
        
        loss_u = np.array([loss_u1, loss_u2])
        loss_uu = 2. * R
        loss_ux = np.zeros((NU, NX))

        return loss, loss_x, loss_xx, loss_u, loss_uu, loss_ux
    
def sim_n_steps(_x0,U,xref,N):
    _U=U.copy()
    X=np.zeros((NX,N+1),dtype=np.float64)
    X[:,0]=_x0
    
    total_grad=0
    loss=0
    for t in range(N):
        x_t=X[:,t]
        u_t=_U[:,t]
        xref_t=xref[:,t]
        x_t1=sim_1_step(x_t,u_t)
        loss += np_quad_form(u_t, R) + np_quad_form(xref_t - x_t, Q)
        #cc=calc_cost(x_t1,u_t,xref_t,t)
        X[:,t+1]=x_t1.copy()
    return X,[loss],U

def sim_1_step(_X,_U):
    
    _state=State(x=_X[0],y=_X[1],v=_X[2],yaw=_X[3])
    nst,Const=Motion.get_next_state(_state,_U[0],_U[1])
    x_t1=nst.array()
    return x_t1#np.array(x_t1,dtype=np.float64)

In [5]:
class iLQRController():
    def __init__(self):
        self.STATE_SIZE=NX
        self.INPUT_SIZE=NU
        self.max_iter=10
        self.dt=0.2
        self.lamb_factor=1.0
        self.lamb_max=10e4
        self.tN=T
        self.eps_converge=0.001
    def ilqr(self, x0, U,xref): 
        #U = np.zeros((NX,T),dtype=np.float64) if U is None else U
        lamb = 1.0 # regularization parameter
        sim_new_trajectory = True
        tN = U.shape[0] # number of time steps
        
        for ii in range(self.max_iter):

            if sim_new_trajectory == True: 
                # simulate forward using the current control trajectory
                X, cost, _ = sim_n_steps(x0, U,xref,T)
                oldcost = copy(cost) # copy for exit condition check

                # 
                f_x = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # df / dx
                f_u = np.zeros((self.tN, self.STATE_SIZE, self.INPUT_SIZE)) # df / du
                # for storing quadratized cost function 

                l = np.zeros((self.tN,1)) # immediate state cost 
                l_x = np.zeros((self.tN, self.STATE_SIZE)) # dl / dx
                l_xx = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # d^2 l / dx^2
                l_u = np.zeros((self.tN, self.INPUT_SIZE)) # dl / du
                l_uu = np.zeros((self.tN, self.INPUT_SIZE, self.INPUT_SIZE)) # d^2 l / du^2
                l_ux = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # d^2 l / du / dx
                # for everything except final state
                for t in range(self.tN):
                    # x(t+1) = f(x(t), u(t)) = x(t) + dx(t) * dt
                    # linearized dx(t) = np.dot(A(t), x(t)) + np.dot(B(t), u(t))
                    # f_x = (np.eye + A(t)) * dt
                    # f_u = (B(t)) * dt
                    # continuous --> discrete
                    A, B, C = Motion.get_linear_model_matrix(X[2,t],X[3,t], U[1,t])
                    f_x[t] = A
                    f_u[t] = B 

                    
                    (l[t], l_x[t], l_xx[t], l_u[t], l_uu[t], l_ux[t]) = calc_cost(X[:,t], U[:,t],xref[:,t],t)
                    

                # and for final state
                #l[-1], l_x[-1], l_xx[-1] = self.cost_final(X[-1])

                sim_new_trajectory = False

            V = l[-1].copy() # value function
            V_x = l_x[-1].copy() # dV / dx
            V_xx = l_xx[-1].copy() # d^2 V / dx^2
            k = np.zeros((self.tN, self.INPUT_SIZE)) # feedforward modification
            K = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # feedback gain

            # work backwards to solve for V, Q, k, and K
            for t in range(self.tN-2, -1, -1):
                
                Q_x = l_x[t] + np.dot(f_x[t].T, V_x) 
                Q_u = l_u[t] + np.dot(f_u[t].T, V_x)

                Q_xx = l_xx[t] + np.dot(f_x[t].T, np.dot(V_xx, f_x[t])) 
                Q_ux = l_ux[t] + np.dot(f_u[t].T, np.dot(V_xx, f_x[t]))
                Q_uu = l_uu[t] + np.dot(f_u[t].T, np.dot(V_xx, f_u[t]))

                Q_uu_evals, Q_uu_evecs = np.linalg.eig(Q_uu)
                Q_uu_evals[Q_uu_evals < 0] = 0.0
                #Q_uu_evals += lamb
                Q_uu_inv = np.dot(Q_uu_evecs, np.dot(np.diag(1.0/Q_uu_evals), Q_uu_evecs.T))
                #k[t]=-1*l_u[t]
                k[t] = -1. * np.dot(Q_uu_inv, Q_u)
                K[t] = -1. * np.dot(Q_uu_inv, Q_ux)

                V_x = Q_x - np.dot(K[t].T, np.dot(Q_uu, k[t]))
                V_xx = Q_xx - np.dot(K[t].T, np.dot(Q_uu, K[t]))

            U_new = np.zeros((NU,T),dtype=np.float64)
            x_new = np.zeros((NX,T),dtype=np.float64)
            x_new = x0.copy()
            for t in range(self.tN - 1): 
                # use feedforward (k) and feedback (K) gain matrices 
                # calculated from our value function approximation
                U_new[:,t] = U[:,t] + k[t] + np.dot(K[t], x_new - X[:,t])
                x_new_1= sim_1_step(x_new, U_new[:,t])
                x_new=x_new_1.copy()
            # evaluate the new trajectory 
            X_new, cost_new , _= sim_n_steps(x0, U_new,xref,T)
            #if 1==1:
            if cost_new[0] < cost[0]: 
                # decrease lambda (get closer to Newton's method)
                lamb /= self.lamb_factor

                X = copy(X_new) # update trajectory 
                U = copy(U_new) # update control signal
                oldcost = copy(cost)
                cost = copy(cost_new)

                sim_new_trajectory = True # do another rollout

                # check to see if update is small enough to exit
                print('...',cost[0],oldcost[0])
                if ii > 0 and ((abs(oldcost[0]-cost[0])/cost[0]) < self.eps_converge):
                    #print("Converged at iteration = %d; Cost = %.4f;"%(ii,cost_new) + 
                    #        " logLambda = %.1f"%np.log(lamb))
                    print("break 1",ii)
                    break

            else: 
                # increase lambda (get closer to gradient descent)
                lamb *= self.lamb_factor
                # print("cost: %.4f, increasing lambda to %.4f")%(cost, lamb)
                if lamb > self.lamb_max: 
                    #print("lambda > max_lambda at iteration = %d;"%ii + 
                    #    " Cost = %.4f; logLambda = %.1f"%(cost, np.log(lamb)))
                    print("break 2",ii)
                    break

        return X, U, cost
    

In [6]:
from Car_Env import Car_Env
from Data_Processor import Data_Processor
from Car_Plot import Move_Contour,Draw_Car

In [7]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
rec=True
max_len=350
qlen=35
car_move=True

Env=Car_Env(rec=rec,max_len=max_len,qlen=qlen,rec_path="Rec_iLQR")
[init_state,map_points]=Env.reset()
mpx=np.array(map_points[:max_len,0])#-ofx
mpy=np.array(map_points[:max_len,1])#-ofy
ix,iy,iyaw,ivel=init_state[0]
_traj=init_state[1][0]
_dind=init_state[1][1]
_end=init_state[1][2]
ofx,ofy,ofang=init_state[2]
img=init_state[3]
state = State(x=ix, y=iy, yaw=iyaw, v=ivel)
#print(init_state)

Track generation: 1234..1551 -> 317-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1225..1535 -> 310-tiles track


In [8]:
if rec==True:
    import os
    if not os.path.exists('Rec_iLQR/'):
        os.mkdir('Rec_iLQR')
    if not os.path.exists('Rec_iLQR_Raw/'):
        os.mkdir('Rec_iLQR_Raw')
    if not os.path.exists('Fig_iLQR/'):
        os.mkdir('Fig_iLQR')


In [ ]:
import cv2
import matplotlib.gridspec as gridspec
import time
phi=0
rot_mat=np.array([[math.cos(phi),math.sin(phi)],[-math.sin(phi),math.cos(phi)]])
dis_time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]

log_state=[]
log_traj=[]
log_img=[]
log_act=[]
log_next_state=[]
log_reward=[]


#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
_index=0
#cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''


_Data_Processor=Data_Processor(_speed=TARGET_SPEED,tlen=qlen)
acc_st=State(x=0,y=0)
itr=0
#fig,axes=plt.subplots(nrows=2,ncols=1,figsize=(10,10))
fig=plt.figure(figsize=(10,10))
gs = fig.add_gridspec(3, 1)
ax1=fig.add_subplot(gs[0,:])
ax2=fig.add_subplot(gs[1:,:])
axes=[ax1,ax2]


def map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _ofang=-1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)

    _vect[0]+=_ofx
    _vect[1]+=_ofy
    return [_vect[0],_vect[1]]
def inv_map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _vect[0]-=_ofx
    _vect[1]-=_ofy
    _ofang=1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)
    return [_vect[0],_vect[1]]
U=np.zeros((NU,T),dtype=np.float64)
iLQR=iLQRController()
init_time=time.time()
while MAX_TIME >= dis_time:
       
        _full_state = np.array([state.x, state.y, state.v, state.yaw],dtype=np.float64)
    
        
        _full_state_traj=_Data_Processor.update_trajectory(_traj)
        xref, dref = _Data_Processor.get_ref_trajectory(state,dl,DT,T)
        
        if _end==True:
            break

        _index+=_dind
        
        #oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(xref, Motion_Status, dref, oa, odelta)
        X,U, cost= iLQR.ilqr(_full_state, U, xref )
        oa=U[0,1]
        odelta=U[1,1]
        ox=X[0,1]
        oy=X[1,1]
        ov=X[2,1]
        oyaw=X[3,1]
        state_str=' x: '+str('%0.4f'%state.x)+' y: '+str('%0.4f'%state.y)+' v: '+str('%0.4f'%state.v)+' yaw: '+str('%0.4f'%np.rad2deg(state.yaw))+'\n'
        xref_str =' x: '+str('%0.4f'%xref[0][0])+' y: '+str('%0.4f'%xref[1][0])+' v: '+str('%0.4f'%xref[2][0])+' yaw: '+str('%0.4f'%np.rad2deg(xref[3][0]))+'\n'
        time_str ='Time(Sec): '+str('%0.4f'%dis_time)+' Sec Distance(m): '+str('%0.4f'%_index)+' m '+'\n'
        #xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        act_str  =' accel: '+str('%0.4f'%accl)+' gas: '+str('%0.4f'%gas)+' steer: '+str('%0.4f'%np.rad2deg(steer))+'\n'
    
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        max_steer=np.deg2rad(45)
        max_accl=5
        if odelta is not None:
            if abs(U[1,0])<max_steer:
                  di= U[1,0]
            else:
                di=(U[1,0]/abs(U[1,0]))*max_steer
            
            if abs(U[0,0])<max_accl:
                  ai=U[0,0]
            else:
                ai=(U[0,0]/abs(U[0,0]))*max_accl
        accl=ai
        steer=di
        
        #act_str=str(steer)+'\n'
        _log_state=[ofx,ofy,_full_state[0],ofang]
        nst,path,offset,img,rw,ter,info=Env.step([steer,accl,0])
        img_raw=Env.render(mode='rgb_array')
        _traj,_dind,_end=path
        ofx,ofy,ofang=offset
        
        gas=Env.gas
        
        state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        _log_next_state=[ofx,ofy,state.v,ofang]
        #Motion_Status.set_state(next_state)
        curr_time=time.time()-init_time
        log_state.append(_log_state)
        log_traj.append(_full_state_traj)
        log_img.append(img)
        log_act.append([accl,steer,0])
        log_next_state.append(_log_next_state)
        log_reward.append([dis_time,curr_time,_index,rw,ter])
        
        dis_time = dis_time + DT
        if car_move==True:
            _xref=xref.copy()
            [_ox,_oy]=map_plot([ox,oy],ofx,ofy,ofang)
            [_xref[0,:],_xref[1,:]]=map_plot([xref[0,:],xref[1,:]],ofx,ofy,ofang)
            [_x,_y]=map_plot([state.x,state.y],ofx,ofy,ofang)
            car_x,car_y,car_ang=ofx,ofy,ofang
            _mpx,_mpy=mpx,mpy
        else:
            _ox,_oy=ox,oy
            _xref=xref
            _x,_y=state.x,state.y
            car_x,car_y,car_ang=state.x,state.y,state.yaw
            [_mpx,_mpy]=inv_map_plot([mpx,mpy],ofx,ofy,ofang)
        x.append(_x)
        y.append(_y)
        yaw.append(ofang)
        v.append(state.v)
        t.append(dis_time)
        d.append(di)
        a.append(ai)

        
    
      
        
        if show_animation:  # pragma: no cover
            axes[0].cla()
            axes[1].cla()
            axes[0].text(.01,.6,'Running : '+time_str,fontsize=10,color='c')
            axes[0].text(.01,.4,'state : '+state_str,fontsize=10,color='r')
            axes[0].text(.01,.2,'xref : '+xref_str,fontsize=10,color='b')
            axes[0].text(.01,0,'act : '+act_str,fontsize=10,color='g')
            
            if ox is not None:
                axes[1].plot(_ox, _oy, "xr", label="MPC",alpha=1.0)
            axes[1].plot(_mpx, _mpy, "-b", label="course")
            axes[1].plot(x, y, "oc", label="trajectory")
            axes[1].plot(_xref[0, :], _xref[1, :], "xk", label="xref",alpha=1.0)
            axes[1].plot(_mpx[_index], _mpy[_index], "xg", label="target",alpha=1.0)
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            car_points=Draw_Car(car_x,car_y)
            rp=Move_Contour(car_points,0,0,car_ang)
            r_car_points=rp.apply()
            r_points=np.array(r_car_points,dtype=np.float64)
            axes[1].plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            #plt.title("Time[s]:" + str(round(time, 2))
            #          + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
            #          + '\nyaw'+str(state.yaw)
            #          + '\nxref_yaw'+str(xref[3][0])
            #          + '\nsteer'+str(steer))
            #fig.canvas.draw()
            plt.pause(0.0001)
            if itr%1==0:
                plt.savefig('Fig_iLQR/'+str(int(itr/1))+'.jpg')
                cv2.imwrite('Rec_iLQR/'+str(int(itr/1))+'.jpg',img)
                cv2.imwrite('Rec_iLQR_Raw/'+str(int(itr/1))+'.jpg',img_raw)
            itr+=1
            if ter==True:
                print('terminal ... ')
                
if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(car_x, car_y, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


... 1651.6745873412929 1673.7489206314144
... 1651.674587315342 1651.6745873412929
break 1 1
... 1650.2077699477923 1650.207774600465
... 1650.207769947792 1650.2077699477923
break 1 1
... 1648.996710175631 1648.9967133025777
... 1648.9967101756308 1648.996710175631
break 1 1
... 1647.7047828253853 1647.7047863489142
... 1646.396449003154 1646.3964525767274
... 1646.3964490031537 1646.396449003154
break 1 1
... 1645.0589058411015 1645.0589095387513
... 1645.058905841101 1645.0589058411015
break 1 1
... 1643.6936866138133 1643.693690427132
... 1642.3003772604584 1642.3003811946533
... 1640.8786958865517 1640.878699944683
... 1640.8786958865512 1640.8786958865517
break 1 1
... 1639.4281480384539 1639.4281522246094
... 1639.4281480384536 1639.4281480384539
break 1 1
... 1637.9489439944123 1637.948948310309
... 1636.44042144568 1636.440425895815
... 1634.902575912753 1634.9025804995165
... 1633.3354145503547 1633.3354192760794
... 1633.3354145503545 1633.3354145503547
break 1 1
... 1631.73

... 1358.9488984752757 1358.9489281916558
... 1358.9488984752707 1358.9488984752757
break 1 1
... 1354.6678565114553 1354.6678866169077
... 1354.66785651145 1354.6678565114553
break 1 1
... 1350.365136209907 1350.3651667047911
... 1350.3651362099017 1350.365136209907
break 1 1
... 1346.0411464195595 1346.041177304582
... 1346.0411464195536 1346.0411464195595
break 1 1
... 1341.6959709015173 1341.6960021809869
... 1341.6959709015114 1341.6959709015173
break 1 1
... 1337.3300239058096 1337.3300555804633
... 1337.3300239058033 1337.3300239058096
break 1 1
... 1332.943472740405 1332.9435048137916
... 1332.9434727403984 1332.943472740405
break 1 1
... 1328.5371255558512 1328.5371580250614
... 1328.5371255558443 1328.5371255558512
break 1 1
... 1324.1112317292875 1324.1112645963995
... 1324.1112317292805 1324.1112317292875
break 1 1
... 1319.6658958056653 1319.6659290749662
... 1319.6658958056576 1319.6658958056653
break 1 1
... 1315.2018082114082 1315.2018418809841
... 1315.2018082114005 13

... 940.6902365038519 940.6903042387236
... 940.6902365038253 940.6902365038519
break 1 1
... 936.2444438937102 936.2445120569864
... 936.2444438936858 936.2444438937102
break 1 1
... 931.8225176058935 931.8225861882852
... 931.8225176058681 931.8225176058935
break 1 1
... 927.4246055562345 927.4246745715745
... 927.4246055562104 927.4246055562345
break 1 1
... 923.0515567606326 923.0516262027799
... 923.0515567606088 923.0515567606326
break 1 1
... 918.7040308217321 918.7041006890178
... 918.7040308217107 918.7040308217321
break 1 1
... 914.3828822838684 914.3829525707501
... 914.382882283846 914.3828822838684
break 1 1
... 910.088604620069 910.0886753301535
... 910.088604620048 910.088604620069
break 1 1
... 905.8221546404425 905.8222257635518
... 905.8221546404218 905.8221546404425
break 1 1
... 901.583423568644 901.5834951292741
... 901.5834235686254 901.583423568644
break 1 1
... 897.3737640584338 897.3738360339248
... 897.3737640584145 897.3737640584338
break 1 1
... 893.19375389

... 793.769526606837 793.7696520941508
... 793.7695266060207 793.769526606837
break 1 1
... 787.8205773290337 787.820703235483
... 787.820577328275 787.8205773290337
break 1 1
... 781.9483824325646 781.9485087656205
... 781.9483824318197 781.9483824325646
break 1 1
... 776.1539075698311 776.1540343279949
... 776.153907569096 776.1539075698311
break 1 1
... 770.4382090240592 770.4383362038673
... 770.4382090232857 770.4382090240592
break 1 1
... 764.8022416274367 764.8023692241679
... 764.8022416267241 764.8022416274367
break 1 1
... 759.2471219880633 759.2472499934706
... 759.2471219874044 759.2471219880633
break 1 1
... 753.7739484016035 753.7740768083916
... 753.7739484009574 753.7739484016035
break 1 1
... 748.3833592216688 748.3834880449435
... 748.3833592209871 748.3833592216688
break 1 1
... 743.0765651222639 743.0766943458221
... 743.0765651216391 743.0765651222639
break 1 1
... 737.8546068119372 737.8547364311768
... 737.8546068113196 737.8546068119372
break 1 1
... 732.7184767

... 682.0972134122568 682.0974088518717
... 682.0972134093835 682.0972134122568
break 1 1
... 674.191124142118 674.191319924757
... 674.1911241392876 674.191124142118
break 1 1
... 666.4340723284458 666.4342684408107
... 666.4340723259262 666.4340723284458
break 1 1
... 658.8271812529705 658.8273776977142
... 658.8271812507597 658.8271812529705
break 1 1
... 651.3720400857037 651.3722368489355
... 651.372040083355 651.3720400857037
break 1 1
... 644.0697118426885 644.0699089204611
... 644.0697118408407 644.0697118426885
break 1 1
... 636.9212987186297 636.9214961229161
... 636.9212987168918 636.9212987186297
break 1 1
... 629.9284046565074 629.9286023573696
... 629.9284046550539 629.9284046565074
break 1 1
... 623.092080697451 623.0922787058762
... 623.0920806960836 623.092080697451
break 1 1
... 616.4135009298209 616.4136992429211
... 616.4135009286855 616.4135009298209
break 1 1
... 609.8942186052986 609.8944172002615
... 609.8942186042419 609.8942186052986
break 1 1
... 603.53514610

...

terminal ... 
... 46.39712553219888 46.39790829536342
... 46.39712552997128 46.39712553219888
break 1 1
terminal ... 
... 48.75898001081679 48.75975923750817
... 48.75898000348281 48.75898001081679
break 1 1
terminal ... 
... 69.55244865221616 69.62557381789733
... 69.5524244069399 69.55244865221616
break 1 1
terminal ... 
... 62.74645043416054 62.74725931289653
... 62.74645033833103 62.74645043416054
break 1 1
terminal ... 
... 56.97796492450616 56.97877673754116
... 56.97796419227934 56.97796492450616
break 1 1
terminal ... 
... 52.22712096656018 52.227928167704704
... 52.22712059519572 52.22712096656018
break 1 1
terminal ... 
... 48.49760185407685 48.49840633616179
... 48.49760161906952 48.49760185407685
break 1 1
terminal ... 
... 45.789528613351905 45.79032832306372
... 45.78952851262873 45.789528613351905
break 1 1
terminal ... 
... 44.10301765404835 44.10381356128826
... 44.10301761633351 44.10301765404835
break 1 1
terminal ... 
... 43.43824203102769 43.439034230195745
... 43.

terminal ... 
... 54.77271355893043 54.773552355393285
... 54.77271355861442 54.77271355893043
break 1 1
terminal ... 
... 48.915061683944785 48.915891273120195
... 48.915061678785804 48.915061683944785
break 1 1
terminal ... 
... 44.12276471131616 44.12359335590821
... 44.122764708974756 44.12276471131616
break 1 1
terminal ... 
... 40.399586176316355 40.40041219062853
... 40.399586175058374 40.399586176316355
break 1 1
terminal ... 
... 37.7450423711904 37.7458660528675
... 37.745042370691706 37.7450423711904
break 1 1
terminal ... 
... 36.15917059997284 36.15999197401954
... 36.159170599813706 36.15917059997284
break 1 1
terminal ... 
... 35.64213603658619 35.64295502534303
... 35.6421360365616 35.64213603658619
break 1 1
terminal ... 
... 36.19391002800536 36.19472660846705
... 36.193910028002996 36.19391002800536
break 1 1
terminal ... 
... 37.81441375000517 37.81522794592052
... 37.814413749986215 37.81441375000517
break 1 1
terminal ... 
... 40.503529527461 40.504341297031345
..

terminal ... 
... 30.34938987935884 30.350239859295097
... 30.349389875391864 30.34938987935884
break 1 1
terminal ... 
... 29.521531779932744 29.52237927456998
... 29.521531778716827 29.521531779932744
break 1 1
terminal ... 
... 29.804609378468093 29.805454377148912
... 29.80460937842528 29.804609378468093
break 1 1
terminal ... 
... 31.19841469355864 31.1992572393343
... 31.198414692710433 31.19841469355864
break 1 1
terminal ... 
... 33.70276834553439 33.70360841217354
... 33.70276834127679 33.70276834553439
break 1 1
terminal ... 
... 37.31737782114402 37.318215481393665
... 37.317377809513886 37.31737782114402
break 1 1
terminal ... 
... 47.0427960599258 47.59496635699304
... 47.01369651348678 47.0427960599258
break 1 1
terminal ... 
... 41.21774453293715 41.223042047414175
... 41.21751617707302 41.21774453293715
break 1 1
terminal ... 
... 36.60008096104522 36.60263624006289
... 36.60003639860631 36.60008096104522
break 1 1
terminal ... 
... 33.07839611397381 33.0800393460523
..

terminal ... 
... 47.05539533947589 49.463636792317374
... 46.96021614340783 47.05539533947589
... 46.95750188124997 46.96021614340783
break 1 2
terminal ... 
... 41.753220723477064 41.756866337536906
... 41.75317044023867 41.753220723477064
break 1 1
terminal ... 
... 38.24400834580264 38.2659754297579
... 38.243791563016146 38.24400834580264
break 1 1
terminal ... 
... 36.30392491815331 36.34799035846128
... 36.30371370053154 36.30392491815331
break 1 1
terminal ... 
... 35.751064320793546 35.840513168035905
... 35.750927297816 35.751064320793546
break 1 1
terminal ... 
... 35.33315696536488 35.39552861064865
... 35.333144953630416 35.33315696536488
break 1 1
terminal ... 
... 35.84055513915659 35.893379124559885
... 35.84046434863425 35.84055513915659
break 1 1
terminal ... 
... 37.69907655968198 37.73603526530824
... 37.69884667331046 37.69907655968198
break 1 1
terminal ... 
... 40.71059005696023 40.73615241547868
... 40.71022605730932 40.71059005696023
break 1 1
terminal ... 
...

terminal ... 
... 34.32026808171832 34.38374413536463
... 34.319623523379164 34.32026808171832
break 1 1
terminal ... 
... 36.706528787867526 36.776704586522946
... 36.70517896932518 36.706528787867526
break 1 1
terminal ... 
... 39.61335166263732 39.673410815965454
... 39.61076936834658 39.61335166263732
break 1 1
terminal ... 
... 43.696441575384654 43.74586584218698
... 43.69184497052303 43.696441575384654
break 1 1
terminal ... 
... 48.9342332578406 48.97778167014706
... 48.92495065129748 48.9342332578406
break 1 1
terminal ... 
... 41.6229273675673 50.76276048008867
... 41.523933903790436 41.6229273675673
... 41.52348796849717 41.523933903790436
break 1 2
terminal ... 
... 38.11895670387286 38.132296509534314
... 38.11885588557282 38.11895670387286
break 1 1
terminal ... 
... 36.420681645442144 36.42894874961815
... 36.42031266789621 36.420681645442144
break 1 1
terminal ... 
... 36.19220559504541 36.20706954637869
... 36.19129770515796 36.19220559504541
break 1 1
terminal ... 
..

In [ ]:
log_dict={
'state':log_state,
'traj':log_traj,
'image':log_img,
'action':log_act,
'next_sate':log_next_state,
'time_reward':log_reward
}

In [ ]:
import pickle
f=open('dataset_iLQR','wb')
pickle.dump(log_dict,f)
f.close()
itr

In [ ]:
f=open('dataset_iLQR','rb')
dataset=pickle.load(f)
f.close()

In [ ]:
dataset.keys()

In [ ]:
traj=np.array(log_traj)#np.array(dataset['traj'])
plt.plot(traj[])
plt.show()

In [ ]:
traj[200]